In [1]:
import sys
sys.path.append('..')
from osp import *

In [2]:
df=get_pmla_df()

100%|██████████| 71902/71902 [00:04<00:00, 17127.13it/s]


In [3]:
df.decade.value_counts()

decade
2000    3104
2010    2813
1970    2556
1990    2541
1980    2513
1950    2172
1960    2106
1930    2105
1940    1956
1920    1605
1910    1233
1900     639
Name: count, dtype: int64

In [4]:
def iter_pmla_sample():
    df = get_pmla_df()
    decs = list(df.groupby('decade'))
    maxn = max(len(decdf) for dec,decdf in decs)
    for i in range(maxn):
        for dec,decdf in decs:
            if len(decdf) > i:
                yield decdf.iloc[i]

In [5]:
row = next(iter_pmla_sample())

100%|██████████| 71902/71902 [00:05<00:00, 14265.30it/s]


In [9]:
def parse_pmla_row(row, force=False):
    id = row['id']
    if not force and id in PMLA_STASH:
        return stanza.Document.from_serialized(PMLA_STASH[id])
    
    text = row['fullText']
    article = filter_pmla_pages(text)
    doc = get_nlp_doc(article)
    PMLA_STASH[id] = doc.to_serialized()
    return doc

In [11]:
doc = parse_pmla_row(row)
tree_stats = pd.DataFrame(get_tree_stats(sent) for sent in doc.sentences)
tree_stats

,num_words,height,num_independent_clauses,num_dependent_clauses,num_words_in_dependent_clauses,num_words_in_independent_clauses,num_punct,num_punct_colon,num_punct_comma,num_parens
0,4,4,0,0,0,4,1,0,0,0
1,40,16,1,2,49,-9,3,0,2,0
2,31,16,4,2,12,19,2,0,1,0
3,35,16,1,2,22,13,4,0,3,0
4,68,15,3,4,84,-16,7,1,3,2
...,...,...,...,...,...,...,...,...,...,...
127,23,9,1,0,0,23,7,0,6,0
128,9,8,1,0,0,9,1,0,0,0
129,29,10,1,0,0,29,3,0,2,0
130,23,11,2,0,0,23,1,0,0,0


In [15]:
tree_stats.mean()

num_words                           31.772727
height                              12.507576
num_independent_clauses              1.363636
num_dependent_clauses                0.878788
num_words_in_dependent_clauses      12.477273
num_words_in_independent_clauses    19.295455
num_punct                            3.856061
num_punct_colon                      0.159091
num_punct_comma                      2.090909
num_parens                           0.568182
dtype: float64

In [16]:
sample = list(iter_pmla_sample())

100%|██████████| 71902/71902 [00:05<00:00, 12495.14it/s]


In [ ]:
for row in tqdm(sample):
    parse_pmla_row(row)

  0%|          | 120/25343 [1:53:43<2219:03:48, 316.72s/it]